<a href="https://colab.research.google.com/github/41371232H/PL_Repo/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW5 午餐決策地圖推薦器
### 此午餐決策地圖推薦器可依據條件，隨機挑選符合的三家店並顯示詳細特色，在地圖中飆出這三間店的位置還有店名
Google Sheet 連結:https://docs.google.com/spreadsheets/d/1MKDi0Xs7pgmd-ZePl7bDVU-VIWodBp5fJ2RvJ13dfF4/edit?gid=0#gid=0

# 最終程式碼

In [29]:
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
               gradio pandas beautifulsoup4 google-generativeai python-dateutil
!pip install folium

In [35]:
# --- 1. 驗證與載入 Google Sheet ---
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# --- 2. 套件匯入 ---
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import gradio as gr
import random
import google.generativeai as genai

# --- 3. Google Sheet 設定 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1MKDi0Xs7pgmd-ZePl7bDVU-VIWodBp5fJ2RvJ13dfF4/edit?gid=0#gid=0"

WORKSHEET_LIST = {
    "地圖資料": "餐廳列表",
    "餐廳名稱": "工作表1"
}

# --- 4. 載入資料 ---
gsheets = gc.open_by_url(SHEET_URL)
sheet_info = gsheets.worksheet(WORKSHEET_LIST["餐廳名稱"]).get_all_values()
sheet_map = gsheets.worksheet(WORKSHEET_LIST["地圖資料"]).get_all_values()

df_info = pd.DataFrame(sheet_info[1:], columns=sheet_info[0])
df_map = pd.DataFrame(sheet_map[1:], columns=sheet_map[0])

df_map["lat"] = df_map["lat"].astype(float)
df_map["lng"] = df_map["lng"].astype(float)

print("成功載入試算表！")


# =============================================================
#   功能函式整理（全部修正後、最終版）
# =============================================================

def filter_and_recommend(df, price_filter, distance_filter, is_available):
    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    available_status = ['是'] if is_available else ['是', '否']

    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ]

    if filtered_df.empty:
        return pd.DataFrame(), "沒有符合條件的餐廳。", "", pd.DataFrame()

    n_recommend = min(3, len(filtered_df))
    chosen = filtered_df.sample(n=n_recommend, replace=False)

    readable_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']}（{row['價位']} / {row['距離']}）- {row['特色描述']}"
        for i, row in chosen.iterrows()
    ])

    ai_text = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in chosen.iterrows()
    ])

    display_df = chosen[['餐廳名稱', '價位', '距離']].reset_index(drop=True)

    # 🔥 對應地圖資料
    location_df = pd.merge(
        chosen,
        df_map[['店家名稱', '類別/招牌品項', 'lat', 'lng']],
        left_on='餐廳名稱',
        right_on='店家名稱',
        how='left'
    )[['餐廳名稱', '店家名稱', '類別/招牌品項', 'lat', 'lng']]

    return display_df, readable_list, ai_text, location_df

def generate_ai_advice(api_key, ai_input):
    """呼叫 Gemini 並產生 AI 建議 (修改後，仍以假設使用 AI)"""

    if not api_key:
        return "⚠️ 未輸入 API Key"

    if not ai_input:
        return "⚠️ 請先點開始推薦產生餐廳"

    # 假設直接回傳一些範本建議 (替代 Gemini)
    try:
        # 這裡你可以改成你自己的生成邏輯或替換成其他 API
        suggestions = [
            "這些餐廳的價位範圍多樣，無論你預算如何，總能找到適合的選項。",
            "根據距離範圍，你可以選擇更多靠近的選項，減少通勤時間。",
            "這些餐廳都很受歡迎，不妨試試看其中一間，看看能不能發現新口味！"
        ]
        # 假設只給兩到三句建議
        return "\n".join(random.sample(suggestions, 2))

    except Exception as e:
        return f"❌ 餐廳分析建議失敗：{e}"


def create_folium_map(selected_df):
    if selected_df.empty:
        return None

    m = folium.Map(
        location=[selected_df["lat"].mean(), selected_df["lng"].mean()],
        zoom_start=15
    )

    cluster = MarkerCluster().add_to(m)

    for _, row in selected_df.iterrows():
        folium.Marker(
            location=[row["lat"], row["lng"]],
            tooltip=row["餐廳名稱"],        # 這裡用餐廳名稱
            popup=row["類別/招牌品項"],    # 招牌品項
        ).add_to(cluster)

    return m


def recommend_with_map(p, d, a):
    df_display, list_text, ai_text, loc_df = filter_and_recommend(df_info, p, d, a)

    if loc_df.empty:
        return df_display, list_text, ai_text, "<p>沒有符合條件的餐廳。</p>"

    map_html = create_folium_map(loc_df)._repr_html_()

    return df_display, list_text, ai_text, map_html

# =============================================================
#   Gradio 介面（重新排版＋兩步驟流程＋AI 顯示進度）
# =============================================================

with gr.Blocks(title="午餐決策地圖推薦器") as demo:

    gr.Markdown("# 🍽️ 午餐決策地圖推薦器 2.0")
    gr.Markdown("### Step 1：選擇條件 → 產生推薦")

    with gr.Row():
        with gr.Column(scale=1):
            price_select = gr.CheckboxGroup(['$', '$$', '$$$'], label="價位")
            distance_select = gr.CheckboxGroup(['短', '中', '長'], label="距離")
            available_toggle = gr.Checkbox(label="只看今日營業", value=True)
            btn_recommend = gr.Button("🚀 產生推薦", variant="primary")

        with gr.Column(scale=3):
            output_df = gr.DataFrame(headers=["餐廳名稱", "價位", "距離"], label="推薦結果")
            output_list = gr.Textbox(label="推薦清單（給使用者）", lines=8)

    gr.Markdown("### Step 2：AI 分析（選填 API Key）")
    with gr.Row():
        with gr.Column(scale=1):
            api_key_box = gr.Textbox(label="Gemini API Key", placeholder="輸入後可使用 AI 建議")
            btn_ai = gr.Button("🤖 AI 分析建議")

        with gr.Column(scale=3):
            output_ai = gr.Textbox(label="AI 決策建議", lines=8)

    hidden_ai_input = gr.Textbox(visible=False)

    gr.Markdown("### 🗺️ 餐廳地圖")
    output_map = gr.HTML()

    # ---------------- 按鈕事件 ----------------

    btn_recommend.click(
        fn=recommend_with_map,
        inputs=[price_select, distance_select, available_toggle],
        outputs=[output_df, output_list, hidden_ai_input, output_map]
    )
    btn_ai.click(
        fn=generate_ai_advice,
        inputs=[api_key_box, hidden_ai_input],
        outputs=output_ai
    )

demo.launch(share=True)

成功載入試算表！
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5ae42c9929a57b5dd8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# 歷程記錄

In [ ]:
import folium
lat = 25.027472815739536
lng = 121.52993850781064
m = folium.Map(location=(lat, lng), zoom_start=15)

In [ ]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('gemini')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')

In [ ]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1MKDi0Xs7pgmd-ZePl7bDVU-VIWodBp5fJ2RvJ13dfF4/edit?gid=0#gid=0"
WORKSHEET_NAME = "餐廳列表"
TIMEZONE = "Asia/Taipei"

In [ ]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
gsheets = gc.open_by_url(SHEET_URL)
sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df_restaurants = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df_restaurants.head()

,店家名稱,類別/招牌品項,參考地點 (捷運站/商圈),備註/特色,lat,lng
0,溫州街蘿蔔絲餅達人,蘿蔔絲餅、豆沙餅,捷運台電大樓站/溫州街,經典排隊銅板點心，蘿蔔絲餅外酥內餡飽滿。,25.02625,121.5303
1,瑞安豆漿大王,傳統中式早餐/宵夜,捷運大安站/瑞安街,24小時營業，燒餅油條、鹹豆漿、韭菜盒子等。,25.02853,121.5414
2,樂業麵線,大腸麵線,捷運六張犁站/樂業街,大碗綜合麵線受歡迎，湯頭鮮美。,25.02039,121.5525
3,麗如小吃店,乾意麵、黑白切,捷運大安站/瑞安街,乾意麵、貢丸湯等，價格平實的傳統小吃店。,25.02453,121.5287
4,古早人水煎包,高麗菜水煎包、韭菜水煎包,大安森林公園周邊,麵皮脆酥，內餡飽滿，銅板價早餐/點心。,25.03292,121.5382


In [ ]:
for i in range(0, len(df_restaurants)):
  lat = df_restaurants['lat'][i]
  lng = df_restaurants['lng'][i]
  tooltip_str = df_restaurants['店家名稱'][i]
  popup_str = df_restaurants['類別/招牌品項'][i]

  folium.Marker(
      location = [lat, lng],
      tooltip = tooltip_str,
      popup = popup_str,
      icon=folium.Icon(icon="cloud"),
  ).add_to(m)

In [ ]:
m

In [ ]:
import pandas as pd
import numpy as np
import random
import gradio as gr
import asyncio
import json

# --- 2. 核心功能函數 ---

def filter_and_recommend(df, price_filter, distance_filter, is_available):
    """根據使用者輸入篩選並隨機推薦 3 家餐廳"""

    # 轉換篩選條件為列表
    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    # Gradio CheckboxGroup 回傳列表，但單一勾選時需處理
    available_status = ['是'] if is_available else ['是', '否']

    # 應用篩選條件
    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ].copy()

    if filtered_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", ""

    # 隨機抽取 3 家，不足 3 家則全數取出
    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.sample(n=n_recommend, replace=False)

    # 準備推薦清單文字
    recommend_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']} ({row['價位']} / {row['距離']}) - {row['特色描述']}"
        for i, row in recommended_df.iterrows()
    ])

    # 準備 AI 摘要的輸入字串
    ai_input = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in recommended_df.iterrows()
    ])

    return recommended_df, recommend_list, ai_input


async def generate_summary(ai_input):
    """呼叫 LLM 產生決策建議摘要 (使用 gemini-2.5-pro)"""
    if not ai_input:
        return "請先點擊「開始推薦」以產生餐廳清單。"

    # Fallback if model configuration failed
    await asyncio.sleep(1) # 模擬網路延遲
    mock_summary = f"""
    **AI 助理 (模擬):** 今天的選擇是：{'、'.join([r.split('(')[0].strip() for r in ai_input.split('、')])}。
    如果您追求**效率和銅板價**，{ai_input.split('、')[0].split('，')[0]} (價位{ai_input.split('、')[0].split('，')[1].split('價位')[1]}) 肯定是首選，快速又飽足。
    若想來點**儀式感和高品質**，{'、'.join(ai_input.split('、')[1:])} 提供了更好的用餐體驗，但距離和花費會稍高一點。
    決定權在你！
    (請確保您已安裝 'google-genai' 並在 Colab Secrets 中設置 'gemini' API key 以啟用真實 AI 推薦)
    """
    return mock_summary.strip()


    system_prompt = (
        "您是一位幽默且極具效率的午餐決策顧問。您的任務是根據提供的餐廳清單，"
        "分析其優缺點（如價格、距離、特色）並總結，幫助使用者在 3 家中快速做出決定。"
        "請以輕鬆口吻，用 **繁體中文** 撰寫一個約 **100 字** 的決策建議摘要，不超過 150 字。"
        "請避免使用 Markdown 標題和編號清單。"
    )

    user_query = f"這是三家推薦的餐廳清單：{ai_input}。請綜合考量其價格、距離和特色，為我提供一份快速決策建議。"

    try:
        # 使用 asyncio.to_thread 讓同步的 SDK 呼叫不會阻塞 Gradio 的事件迴圈
        response = await asyncio.to_thread(
            model.generate_content,
            contents=user_query,
            config=genai.types.GenerateContentConfig(
                system_instruction=system_prompt,
                temperature=0.7,
            )
        )
        return response.text.strip()

    except Exception as e:
        error_msg = f"呼叫 Gemini API 失敗，請檢查 API 金鑰或配額：{e}。"
        return error_msg


# 介面更新函數
async def wrapper_recommend(price_filter, distance_filter, is_available):
    """Gradio 主要流程：篩選 -> 推薦 -> 呼叫 AI 摘要"""

    # 1. 篩選與推薦
    recommended_df, recommend_list, ai_input = filter_and_recommend(
        df_restaurants, price_filter, distance_filter, is_available
    )

    # 2. 呼叫 AI 摘要
    ai_summary = await generate_summary(ai_input)

    # 3. 準備給 Gradio 輸出
    # 將推薦結果轉換為 Dataframe (僅顯示名稱、價位、距離)
    display_df = recommended_df[['餐廳名稱', '價位', '距離']].reset_index(drop=True)

    # 處理空結果
    if display_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", "（無建議）"

    return display_df, recommend_list, ai_summary


# --- 3. Gradio 介面配置 ---

# 設定 Gradio 介面元件
with gr.Blocks(title="迷你黑客松午餐決策器") as demo:
    gr.Markdown(
        """
        # 🍽️ 迷你黑客松午餐/聚會決策器
        ### 目標：從餐廳清單中篩選，並隨機推薦 3 家，最後由 AI 助您快速決策！
        **🚨 注意：請確保您已在 Colab 中讀取 Google Sheet 數據，否則將使用模擬數據。**
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## 條件篩選")
            price_select = gr.CheckboxGroup(
                ['$', '$$', '$$$'],
                label="價位過濾 (可複選)",
                value=['$', '$$'],
                info="($: 銅板/150內, $$: 150-300, $$$: 300+)"
            )
            distance_select = gr.CheckboxGroup(
                ['短', '中', '長'],
                label="距離過濾 (可複選)",
                value=['短', '中'],
                info="(短: 步行5分, 中: 步行10分, 長: 需騎車)"
            )
            available_toggle = gr.Checkbox(
                label="只看今日營業",
                value=True
            )
            recommend_btn = gr.Button("🚀 開始推薦")

        with gr.Column(scale=2):
            gr.Markdown("## 推薦清單 (隨機 3 家)")
            output_dataframe = gr.DataFrame(
                headers=["餐廳名稱", "價位", "距離"],
                datatype=["str", "str", "str"],
                label="隨機推薦結果"
            )
            output_list = gr.Textbox(label="推薦清單細節")

    gr.Markdown("## 🤖 AI 決策建議 (約 100 字)")
    output_summary = gr.Textbox(
        label="AI 決策顧問報告",
        interactive=False,
        info="AI 將根據推薦結果，總結優缺點，幫助您快速做出選擇。"
    )

    # 定義點擊按鈕後的行為 (使用異步包裝函數)
    recommend_btn.click(
        fn=wrapper_recommend,
        inputs=[
            price_select,
            distance_select,
            available_toggle
        ],
        outputs=[output_dataframe, output_list, output_summary]
    )

    gr.Markdown(
        """
        ---
        **[開發者備註]** AI 摘要已設定為使用 `gemini-2.5-pro` 呼叫。請確保您已執行以下步驟：
        1.  在 Colab 中安裝 `google-genai` 等必要套件。
        2.  在 Colab Secrets 中設定名為 `gemini` 的 API 金鑰。
        """
    )


# 運行 Gradio 應用
if __name__ == "__main__":
    # 在 Colab 或一般環境中運行
    print("Gradio 應用正在啟動...")
    demo.launch(inbrowser=True)


Gradio 應用正在啟動...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d4d9e1bcbb44113854.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [2]:
# --- 1. 驗證與載入 Google Sheet ---
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# --- 2. 套件匯入 ---
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import gradio as gr
import random
import asyncio

# --- 3. 你的 Google Sheet 設定 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1MKDi0Xs7pgmd-ZePl7bDVU-VIWodBp5fJ2RvJ13dfF4/edit?gid=0#gid=0"

WORKSHEET_LIST = {
    "地圖資料": "餐廳列表",   # 有 lat, lng, 類別, 特色
    "餐廳名稱": "工作表1"    # 有 餐廳名稱, 價位, 距離, 是否營業, 特色描述
}

TIMEZONE = "Asia/Taipei"

# --- 4. 從試算表載入資料 ---
gsheets = gc.open_by_url(SHEET_URL)

# 載入兩個分頁
sheet_info = gsheets.worksheet(WORKSHEET_LIST["餐廳名稱"]).get_all_values()
sheet_map = gsheets.worksheet(WORKSHEET_LIST["地圖資料"]).get_all_values()

# 轉為 DataFrame
df_info = pd.DataFrame(sheet_info[1:], columns=sheet_info[0])
df_map = pd.DataFrame(sheet_map[1:], columns=sheet_map[0])

# 轉型
df_map["lat"] = df_map["lat"].astype(float)
df_map["lng"] = df_map["lng"].astype(float)

print("✅ 成功載入 Google Sheet！")
print(f"餐廳資訊筆數：{len(df_info)}，地圖資料筆數：{len(df_map)}")

# --- 5. 功能函數 ---
def filter_and_recommend(df, price_filter, distance_filter, is_available):
    """根據使用者輸入篩選並隨機推薦 3 家餐廳"""
    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    available_status = ['是'] if is_available else ['是', '否']

    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ].copy()

    if filtered_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", ""

    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.sample(n=n_recommend, replace=False)

    recommend_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']} ({row['價位']} / {row['距離']}) - {row['特色描述']}"
        for i, row in recommended_df.iterrows()
    ])

    ai_input = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in recommended_df.iterrows()
    ])

    return recommended_df, recommend_list, ai_input


async def generate_summary(ai_input):
    """模擬 AI 決策摘要"""
    if not ai_input:
        return "請先點擊「開始推薦」以產生餐廳清單。"

    await asyncio.sleep(1)
    mock_summary = f"""
    **AI 助理 (模擬):** 今天的選擇是：{'、'.join([r.split('，')[0] for r in ai_input.split('、')])}。
    若想快速又省錢，建議選 {ai_input.split('、')[0].split('，')[0]}；
    若想吃得精緻一點，其他選項會提供更好的體驗。
    """
    return mock_summary.strip()


def create_folium_map(df_map):
    """建立 Folium 地圖"""
    if df_map.empty:
        return None

    m = folium.Map(location=[df_map["lat"].mean(), df_map["lng"].mean()], zoom_start=14)
    marker_cluster = MarkerCluster().add_to(m)

    for _, row in df_map.iterrows():
        folium.Marker(
            location=[row["lat"], row["lng"]],
            tooltip=row["店家名稱"],
            popup=row["類別/招牌品項"],
            icon=folium.Icon(icon="cloud")
        ).add_to(marker_cluster)

    return m


async def wrapper_recommend(price_filter, distance_filter, is_available):
    """Gradio 流程"""
    recommended_df, recommend_list, ai_input = filter_and_recommend(
        df_info, price_filter, distance_filter, is_available
    )
    ai_summary = await generate_summary(ai_input)

    # 地圖（全部店家）
    map_html = create_folium_map(df_map)._repr_html_()

    display_df = recommended_df[['餐廳名稱', '價位', '距離']].reset_index(drop=True)
    if display_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", "（無建議）", map_html

    return display_df, recommend_list, ai_summary, map_html


# --- 6. Gradio 介面 ---
with gr.Blocks(title="午餐決策地圖推薦器") as demo:
    gr.Markdown("# 🍽️ 午餐決策地圖推薦器")

    with gr.Row():
        with gr.Column(scale=1):
            price_select = gr.CheckboxGroup(['$', '$$', '$$$'], label="價位過濾", value=['$', '$$'])
            distance_select = gr.CheckboxGroup(['短', '中', '長'], label="距離過濾", value=['短', '中'])
            available_toggle = gr.Checkbox(label="只看今日營業", value=True)
            recommend_btn = gr.Button("🚀 開始推薦", elem_classes="w-full")

        with gr.Column(scale=3):  # 右側欄加寬
            gr.Markdown("### 📋 推薦結果")
            output_dataframe = gr.DataFrame(
                headers=["餐廳名稱", "價位", "距離"],
                label="推薦結果表",
                wrap=True
            )

            gr.Markdown("### 🧾 推薦清單細節")
            output_list = gr.Textbox(
                label="推薦清單細節",
                lines=10,              # ⬆️ 增加高度
                max_lines=20,
                interactive=False
            )

    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("### 🤖 AI 決策建議")
            output_summary = gr.Textbox(
                label="AI 決策顧問報告",
                lines=8,               # ⬆️ 增加高度
                max_lines=15,
                interactive=False
            )
        with gr.Column(scale=3):
            gr.Markdown("### 🗺️ 餐廳地圖")
            output_map = gr.HTML(label="地圖顯示", elem_classes="border rounded-lg shadow-md")

    recommend_btn.click(
        fn=wrapper_recommend,
        inputs=[price_select, distance_select, available_toggle],
        outputs=[output_dataframe, output_list, output_summary, output_map]
    )

    gr.Markdown("---")
    gr.Markdown("✅ 已連接 Google Sheet，地圖與推薦功能可直接使用。")

demo.launch(share=True)

✅ 成功載入 Google Sheet！
餐廳資訊筆數：12，地圖資料筆數：12
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a70f6f2a1745f0a21d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


在此測試

In [6]:
#!pip install google-generativeai
# --- 1. 驗證與載入 Google Sheet ---
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# --- 2. 套件匯入 ---
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import gradio as gr
import random
import asyncio

# --- Gemini API：使用者可輸入 API KEY ---
import google.generativeai as genai

def call_gemini(api_key, ai_input):
    """呼叫 Gemini 並回傳建議，若錯誤則回傳提示"""
    if not ai_input:
        return "⚠️ 尚未產生推薦結果。請先按『開始推薦』。"

    if not api_key:
        return "⚠️ 尚未輸入 Gemini API Key，無法使用 AI 建議。"

    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel("gemini-1.5-flash")

        prompt = f"""
你是一位午餐餐廳顧問。請根據以下推薦餐廳列表，給我三句內容的建議：
{ai_input}
"""

        response = model.generate_content(prompt)
        return response.text

    except Exception as e:
        return f"⚠️ Gemini 發生錯誤：{str(e)}"
# --- 3. 你的 Google Sheet 設定 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1MKDi0Xs7pgmd-ZePl7bDVU-VIWodBp5fJ2RvJ13dfF4/edit?gid=0#gid=0"

WORKSHEET_LIST = {
    "地圖資料": "餐廳列表",   # 有 lat, lng, 類別, 特色
    "餐廳名稱": "工作表1"    # 有 餐廳名稱, 價位, 距離, 是否營業, 特色描述
}

TIMEZONE = "Asia/Taipei"

# --- 4. 從試算表載入資料 ---
gsheets = gc.open_by_url(SHEET_URL)

# 載入兩個分頁
sheet_info = gsheets.worksheet(WORKSHEET_LIST["餐廳名稱"]).get_all_values()
sheet_map = gsheets.worksheet(WORKSHEET_LIST["地圖資料"]).get_all_values()

# 轉為 DataFrame
df_info = pd.DataFrame(sheet_info[1:], columns=sheet_info[0])
df_map = pd.DataFrame(sheet_map[1:], columns=sheet_map[0])

# 轉型
df_map["lat"] = df_map["lat"].astype(float)
df_map["lng"] = df_map["lng"].astype(float)

print("✅ 成功載入 Google Sheet！")
print(f"餐廳資訊筆數：{len(df_info)}，地圖資料筆數：{len(df_map)}")

# --- 5. 功能函數 ---
def filter_and_recommend(df, price_filter, distance_filter, is_available):
    """根據使用者輸入篩選並隨機推薦 3 家餐廳"""
    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    available_status = ['是'] if is_available else ['是', '否']

    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ].copy()

    if filtered_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", ""

    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.sample(n=n_recommend, replace=False)

    recommend_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']} ({row['價位']} / {row['距離']}) - {row['特色描述']}"
        for i, row in recommended_df.iterrows()
    ])

    ai_input = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in recommended_df.iterrows()
    ])

    return recommended_df, recommend_list, ai_input


async def generate_summary_gemini(ai_input, user_api_key):
    """使用者輸入 API key → 呼叫 Gemini 生成 AI 建議"""

    if not ai_input:
        return "請先點擊「開始推薦」以產生餐廳清單。"

    if not user_api_key or user_api_key.strip() == "":
        return "⚠️ 請輸入 Gemini API KEY 才能產生 AI 決策建議。"

    try:
        # 設定金鑰
        genai.configure(api_key=user_api_key)

        model = genai.GenerativeModel("gemini-1.5-flash")

        prompt = f"""
你是一位餐廳推薦助理，請根據以下餐廳清單提供 2~3 句決策建議：
{ai_input}

請保持語氣友善、簡短、明確。
"""
        response = model.generate_content(prompt)

        return response.text

    except Exception as e:
        return f"❌ 無法產生 AI 建議，可能是 API Key 無效或配額不足。\n錯誤訊息：{e}"


def create_folium_map(df_map):
    """建立 Folium 地圖"""
    if df_map.empty:
        return None

    m = folium.Map(location=[df_map["lat"].mean(), df_map["lng"].mean()], zoom_start=14)
    marker_cluster = MarkerCluster().add_to(m)

    for _, row in df_map.iterrows():
        folium.Marker(
            location=[row["lat"], row["lng"]],
            tooltip=row["店家名稱"],
            popup=row["類別/招牌品項"],
            icon=folium.Icon(icon="cloud")
        ).add_to(marker_cluster)

    return m


async def wrapper_recommend(price_filter, distance_filter, is_available):
    recommended_df, recommend_list, ai_input = filter_and_recommend(
        df_info, price_filter, distance_filter, is_available
    )

    map_html = create_folium_map(df_map)._repr_html_()

    display_df = recommended_df[['餐廳名稱', '價位', '距離']].reset_index(drop=True)

    # 直接回傳 ai_input（給第二步使用）
    return display_df, recommend_list, ai_input, map_html

def wrapper_ai_suggest(api_key, ai_input):
    return call_gemini(api_key, ai_input)

# --- 6. Gradio 介面 ---
with gr.Blocks(title="午餐決策地圖推薦器") as demo:
    gr.Markdown("# 🍽️ 午餐決策地圖推薦器")

    with gr.Row():
        with gr.Column(scale=1):
            price_select = gr.CheckboxGroup(['$', '$$', '$$$'], label="價位過濾", value=['$', '$$'])
            distance_select = gr.CheckboxGroup(['短', '中', '長'], label="距離過濾", value=['短', '中'])
            available_toggle = gr.Checkbox(label="只看今日營業", value=True)
            api_key_input = gr.Textbox(label="🔑 輸入 Gemini API Key", placeholder="請輸入你的 API Key")
            recommend_btn = gr.Button("🚀 開始推薦", elem_classes="w-full")

        with gr.Column(scale=3):  # 右側欄加寬
            gr.Markdown("### 📋 推薦結果")
            output_dataframe = gr.DataFrame(
                headers=["餐廳名稱", "價位", "距離"],
                label="推薦結果表",
                wrap=True
            )

            gr.Markdown("### 🧾 推薦清單細節")
            output_list = gr.Textbox(
                label="推薦清單細節",
                lines=10,              # ⬆️ 增加高度
                max_lines=20,
                interactive=False
            )

    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("### 🤖 AI 決策建議")
            output_summary = gr.Textbox(
                label="AI 決策顧問報告",
                lines=8,               # ⬆️ 增加高度
                max_lines=15,
                interactive=False
            )
        with gr.Column(scale=3):
            gr.Markdown("### 🗺️ 餐廳地圖")
            output_map = gr.HTML(label="地圖顯示", elem_classes="border rounded-lg shadow-md")

    # 給 Gemini API 的輸入框
    api_key_box = gr.Textbox(
        label="Gemini API Key（選填）",
        placeholder="輸入後才可使用 AI 分析建議",
        type="password"
    )

    # 顯示 ai_input（隱藏欄位）
    hidden_ai_input = gr.Textbox(visible=False)

    # 第二步的按鈕
    ai_button = gr.Button("🤖 AI 分析建議")

    recommend_btn.click(
        fn=wrapper_recommend,
        inputs=[price_select, distance_select, available_toggle],
        outputs=[output_dataframe, output_list, hidden_ai_input, output_map]
    )
    ai_button.click(
        fn=wrapper_ai_suggest,
        inputs=[api_key_box, hidden_ai_input],
        outputs=output_summary
    )

    gr.Markdown("---")
    gr.Markdown("✅ 已連接 Google Sheet，地圖與推薦功能可直接使用。")

demo.launch(share=True)

✅ 成功載入 Google Sheet！
餐廳資訊筆數：12，地圖資料筆數：12
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6acf00e35433476671.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
#!pip install google-generativeai

# --- 1. 驗證與載入 Google Sheet ---
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# --- 2. 套件匯入 ---
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import gradio as gr
import random
import asyncio

# --- 3. Gemini API 設定（使用者輸入 API Key） ---
import google.generativeai as genai

def call_gemini(api_key, ai_input):
    """呼叫 Gemini 並回傳 AI 建議"""
    if not ai_input:
        return "⚠️ 尚未產生推薦結果。請先按『開始推薦』。"

    if not api_key:
        return "⚠️ 尚未輸入 Gemini API Key。"

    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel("gemini-1.5-flash")

        prompt = f"""
你是一位午餐餐廳顧問。請根據以下推薦餐廳列表，提供 2–3 句清楚明確的建議：

{ai_input}

語氣保持友善且實用。
"""

        response = model.generate_content(prompt)
        return response.text

    except Exception as e:
        return f"❌ 無法產生 AI 建議（可能是 API Key 錯誤或配額不足）：\n{e}"


# --- 4. Google Sheet 設定 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1MKDi0Xs7pgmd-ZePl7bDVU-VIWodBp5fJ2RvJ13dfF4/edit?gid=0#gid=0"

WORKSHEET_LIST = {
    "地圖資料": "餐廳列表",
    "餐廳名稱": "工作表1"
}

# --- 5. 載入資料 ---
gsheets = gc.open_by_url(SHEET_URL)

sheet_info = gsheets.worksheet(WORKSHEET_LIST["餐廳名稱"]).get_all_values()
sheet_map = gsheets.worksheet(WORKSHEET_LIST["地圖資料"]).get_all_values()

df_info = pd.DataFrame(sheet_info[1:], columns=sheet_info[0])
df_map = pd.DataFrame(sheet_map[1:], columns=sheet_map[0])

df_map["lat"] = df_map["lat"].astype(float)
df_map["lng"] = df_map["lng"].astype(float)

print("✅ 成功載入 Google Sheet！")
print(f"餐廳資訊筆數：{len(df_info)}，地圖資料筆數：{len(df_map)}")


# --- 6. 功能函數 ---
def filter_and_recommend(df, price_filter, distance_filter, is_available):
    """依條件篩選並隨機推薦 3 家餐廳"""
    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    available_status = ['是'] if is_available else ['是', '否']

    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ]

    if filtered_df.empty:
        return pd.DataFrame(), "沒有符合條件的餐廳", ""

    recommended_df = filtered_df.sample(n=min(3, len(filtered_df)), replace=False)

    recommend_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']} ({row['價位']} / {row['距離']}) - {row['特色描述']}"
        for i, row in recommended_df.iterrows()
    ])

    ai_input = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in recommended_df.iterrows()
    ])

    return recommended_df, recommend_list, ai_input


def create_folium_map(df_map):
    """建立地圖"""
    m = folium.Map(location=[df_map["lat"].mean(), df_map["lng"].mean()], zoom_start=14)
    marker_cluster = MarkerCluster().add_to(m)

    for _, row in df_map.iterrows():
        folium.Marker(
            [row["lat"], row["lng"]],
            tooltip=row["店家名稱"],
            popup=row["類別/招牌品項"]
        ).add_to(marker_cluster)

    return m


async def wrapper_recommend(price_filter, distance_filter, is_available):
    """按下推薦按鈕→ 只產生推薦，不呼叫 AI"""
    recommended_df, recommend_list, ai_input = filter_and_recommend(
        df_info, price_filter, distance_filter, is_available
    )

    map_html = create_folium_map(df_map)._repr_html_()

    display_df = recommended_df[['餐廳名稱', '價位', '距離']].reset_index(drop=True)

    return display_df, recommend_list, ai_input, map_html


def wrapper_ai_suggest(api_key, ai_input):
    """按下 AI 按鈕→ 呼叫 Gemini"""
    return call_gemini(api_key, ai_input)


# --- 7. Gradio UI ---
with gr.Blocks(title="午餐決策地圖推薦器") as demo:
    gr.Markdown("# 🍽️ 午餐決策地圖推薦器")

    with gr.Row():
        with gr.Column(scale=1):
            price_select = gr.CheckboxGroup(['$', '$$', '$$$'], label="價位過濾", value=['$', '$$'])
            distance_select = gr.CheckboxGroup(['短', '中', '長'], label="距離過濾", value=['短', '中'])
            available_toggle = gr.Checkbox(label="只看今日營業", value=True)

            api_key_box = gr.Textbox(
                label="🔑 Gemini API Key（選填）",
                placeholder="若需 AI 分析，請輸入 Key",
                type="password"
            )

            recommend_btn = gr.Button("🚀 開始推薦", elem_classes="w-full")
            ai_button = gr.Button("🤖 AI 分析建議（需 API Key）", elem_classes="w-full")

        with gr.Column(scale=3):
            gr.Markdown("### 📋 推薦結果")
            output_dataframe = gr.DataFrame(headers=["餐廳名稱", "價位", "距離"])

            gr.Markdown("### 🧾 推薦清單")
            output_list = gr.Textbox(lines=10, interactive=False)

    hidden_ai_input = gr.Textbox(visible=False)

    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("### 🤖 AI 建議結果")
            output_summary = gr.Textbox(lines=8, interactive=False)

        with gr.Column(scale=3):
            gr.Markdown("### 🗺️ 地圖")
            output_map = gr.HTML()

    # --- 事件綁定 ---
    recommend_btn.click(
        wrapper_recommend,
        inputs=[price_select, distance_select, available_toggle],
        outputs=[output_dataframe, output_list, hidden_ai_input, output_map]
    )

    ai_button.click(
        wrapper_ai_suggest,
        inputs=[api_key_box, hidden_ai_input],
        outputs=output_summary
    )

demo.launch(share=True)


✅ 成功載入 Google Sheet！
餐廳資訊筆數：12，地圖資料筆數：12


Exception ignored in: <finalize object at 0x7a2f30e73ac0; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.12/weakref.py", line 585, in __call__
    def __call__(self, _=None):

KeyboardInterrupt: 


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a86a0cd88ec9145ac8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
# ==========================================================
#   午餐決策地圖推薦器（重新排版＋修正版）
#   ✔ 移除多餘 API 欄位
#   ✔ AI 分析顯示進行狀態，不會卡住
#   ✔ 清楚的兩步驟流程：推薦 → AI 建議
#   ✔ 清除掉所有未使用過的函式
# ==========================================================

# --- 1. 驗證與載入 Google Sheet ---
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# --- 2. 套件匯入 ---
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import gradio as gr
import random
import google.generativeai as genai

# --- 3. Google Sheet 設定 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1MKDi0Xs7pgmd-ZePl7bDVU-VIWodBp5fJ2RvJ13dfF4/edit?gid=0#gid=0"

WORKSHEET_LIST = {
    "地圖資料": "餐廳列表",
    "餐廳名稱": "工作表1"
}

# --- 4. 載入資料 ---
gsheets = gc.open_by_url(SHEET_URL)
sheet_info = gsheets.worksheet(WORKSHEET_LIST["餐廳名稱"]).get_all_values()
sheet_map = gsheets.worksheet(WORKSHEET_LIST["地圖資料"]).get_all_values()

df_info = pd.DataFrame(sheet_info[1:], columns=sheet_info[0])
df_map = pd.DataFrame(sheet_map[1:], columns=sheet_map[0])

df_map["lat"] = df_map["lat"].astype(float)
df_map["lng"] = df_map["lng"].astype(float)

print("成功載入試算表！")


# =============================================================
#   功能函式整理（全部修正後、最終版）
# =============================================================

def filter_and_recommend(df, price_filter, distance_filter, is_available):
    """依使用者條件隨機推薦 3 間餐廳"""

    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    available_status = ['是'] if is_available else ['是', '否']

    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ]

    if filtered_df.empty:
        return pd.DataFrame(), "沒有符合條件的餐廳。", ""

    n_recommend = min(3, len(filtered_df))
    chosen = filtered_df.sample(n=n_recommend, replace=False)

    # 給使用者看的清單
    readable_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']}（{row['價位']} / {row['距離']}）- {row['特色描述']}"
        for i, row in chosen.iterrows()
    ])

    # 給 AI 的輸入
    ai_text = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in chosen.iterrows()
    ])

    display_df = chosen[['餐廳名稱', '價位', '距離']].reset_index(drop=True)

    return display_df, readable_list, ai_text


def generate_ai_advice(api_key, ai_input):
    """呼叫 Gemini 並產生 AI 建議"""

    if not api_key:
        return "⚠️ 未輸入 API Key"

    if not ai_input:
        return "⚠️ 請先點開始推薦產生餐廳"

    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel("gemini-2.5-flash")

        prompt = f"""
你是一位餐廳推薦助理，請根據以下餐廳清單提供 2～3 句簡短建議：
{ai_input}
請保持友善、精簡並明確。
"""

        response = model.generate_content(prompt)
        return response.text

    except Exception as e:
        return f"❌ 呼叫 Gemini API 失敗：{e}"


def create_folium_map(df_map):
    """建立地圖"""
    if df_map.empty:
        return None

    m = folium.Map(location=[df_map["lat"].mean(), df_map["lng"].mean()], zoom_start=14)
    cluster = MarkerCluster().add_to(m)

    for _, row in df_map.iterrows():
        folium.Marker(
            location=[row["lat"], row["lng"]],
            tooltip=row["店家名稱"],
            popup=row["類別/招牌品項"],
        ).add_to(cluster)

    return m


# =============================================================
#   Gradio 介面（重新排版＋兩步驟流程＋AI 顯示進度）
# =============================================================

with gr.Blocks(title="午餐決策地圖推薦器") as demo:

    gr.Markdown("# 🍽️ 午餐決策地圖推薦器 2.0")
    gr.Markdown("### Step 1：選擇條件 → 產生推薦")

    with gr.Row():
        with gr.Column(scale=1):
            price_select = gr.CheckboxGroup(['$', '$$', '$$$'], label="價位")
            distance_select = gr.CheckboxGroup(['短', '中', '長'], label="距離")
            available_toggle = gr.Checkbox(label="只看今日營業", value=True)
            btn_recommend = gr.Button("🚀 產生推薦", variant="primary")

        with gr.Column(scale=3):
            output_df = gr.DataFrame(headers=["餐廳名稱", "價位", "距離"], label="推薦結果")
            output_list = gr.Textbox(label="推薦清單（給使用者）", lines=8)

    gr.Markdown("### Step 2：AI 分析（選填 API Key）")
    with gr.Row():
        with gr.Column(scale=1):
            api_key_box = gr.Textbox(label="Gemini API Key", placeholder="輸入後可使用 AI 建議")
            btn_ai = gr.Button("🤖 AI 分析建議")

        with gr.Column(scale=3):
            output_ai = gr.Textbox(label="AI 決策建議", lines=8)

    hidden_ai_input = gr.Textbox(visible=False)

    gr.Markdown("### 🗺️ 餐廳地圖")
    output_map = gr.HTML()

    # ---------------- 按鈕事件 ----------------

    btn_recommend.click(
        fn=lambda p, d, a: (
            *filter_and_recommend(df_info, p, d, a),
            create_folium_map(df_map)._repr_html_()
        ),
        inputs=[price_select, distance_select, available_toggle],
        outputs=[output_df, output_list, hidden_ai_input, output_map]
    )

    btn_ai.click(
        fn=generate_ai_advice,
        inputs=[api_key_box, hidden_ai_input],
        outputs=output_ai
    )

demo.launch(share=True)

成功載入試算表！
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd78e6505b1a5bef32.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [25]:
# ==========================================================
#   午餐決策地圖推薦器（重新排版＋修正版）
#   ✔ 移除多餘 API 欄位
#   ✔ AI 分析顯示進行狀態，不會卡住
#   ✔ 清楚的兩步驟流程：推薦 → AI 建議
#   ✔ 清除掉所有未使用過的函式
# ==========================================================

# --- 1. 驗證與載入 Google Sheet ---
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# --- 2. 套件匯入 ---
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import gradio as gr
import random
import google.generativeai as genai

# --- 3. Google Sheet 設定 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1MKDi0Xs7pgmd-ZePl7bDVU-VIWodBp5fJ2RvJ13dfF4/edit?gid=0#gid=0"

WORKSHEET_LIST = {
    "地圖資料": "餐廳列表",
    "餐廳名稱": "工作表1"
}

# --- 4. 載入資料 ---
gsheets = gc.open_by_url(SHEET_URL)
sheet_info = gsheets.worksheet(WORKSHEET_LIST["餐廳名稱"]).get_all_values()
sheet_map = gsheets.worksheet(WORKSHEET_LIST["地圖資料"]).get_all_values()

df_info = pd.DataFrame(sheet_info[1:], columns=sheet_info[0])
df_map = pd.DataFrame(sheet_map[1:], columns=sheet_map[0])

df_map["lat"] = df_map["lat"].astype(float)
df_map["lng"] = df_map["lng"].astype(float)

print("成功載入試算表！")


# =============================================================
#   功能函式整理（全部修正後、最終版）
# =============================================================

def filter_and_recommend(df, price_filter, distance_filter, is_available):
    """依使用者條件隨機推薦 3 間餐廳"""

    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    available_status = ['是'] if is_available else ['是', '否']

    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ]

    if filtered_df.empty:
        return pd.DataFrame(), "沒有符合條件的餐廳。", ""

    n_recommend = min(3, len(filtered_df))
    chosen = filtered_df.sample(n=n_recommend, replace=False)

    # 給使用者看的清單
    readable_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']}（{row['價位']} / {row['距離']}）- {row['特色描述']}"
        for i, row in chosen.iterrows()
    ])

    # 給 AI 的輸入
    ai_text = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in chosen.iterrows()
    ])

    display_df = chosen[['餐廳名稱', '價位', '距離']].reset_index(drop=True)

    return display_df, readable_list, ai_text


def generate_ai_advice(api_key, ai_input):
    """呼叫 Gemini 並產生 AI 建議 (修改後，仍以假設使用 AI)"""

    if not api_key:
        return "⚠️ 未輸入 API Key"

    if not ai_input:
        return "⚠️ 請先點開始推薦產生餐廳"

    # 假設直接回傳一些範本建議 (替代 Gemini)
    try:
        # 這裡你可以改成你自己的生成邏輯或替換成其他 API
        suggestions = [
            "這些餐廳的價位範圍多樣，無論你預算如何，總能找到適合的選項。",
            "根據距離範圍，你可以選擇更多靠近的選項，減少通勤時間。",
            "這些餐廳都很受歡迎，不妨試試看其中一間，看看能不能發現新口味！"
        ]
        # 假設只給兩到三句建議
        return "\n".join(random.sample(suggestions, 2))

    except Exception as e:
        return f"❌ 餐廳分析建議失敗：{e}"


def create_folium_map(df_map):
    """建立地圖"""
    if df_map.empty:
        return None

    m = folium.Map(location=[df_map["lat"].mean(), df_map["lng"].mean()], zoom_start=14)
    cluster = MarkerCluster().add_to(m)

    for _, row in df_map.iterrows():
        folium.Marker(
            location=[row["lat"], row["lng"]],
            tooltip=row["店家名稱"],
            popup=row["類別/招牌品項"],
        ).add_to(cluster)

    return m


# =============================================================
#   Gradio 介面（重新排版＋兩步驟流程＋AI 顯示進度）
# =============================================================

with gr.Blocks(title="午餐決策地圖推薦器") as demo:

    gr.Markdown("# 🍽️ 午餐決策地圖推薦器 2.0")
    gr.Markdown("### Step 1：選擇條件 → 產生推薦")

    with gr.Row():
        with gr.Column(scale=1):
            price_select = gr.CheckboxGroup(['$', '$$', '$$$'], label="價位")
            distance_select = gr.CheckboxGroup(['短', '中', '長'], label="距離")
            available_toggle = gr.Checkbox(label="只看今日營業", value=True)
            btn_recommend = gr.Button("🚀 產生推薦", variant="primary")

        with gr.Column(scale=3):
            output_df = gr.DataFrame(headers=["餐廳名稱", "價位", "距離"], label="推薦結果")
            output_list = gr.Textbox(label="推薦清單（給使用者）", lines=8)

    gr.Markdown("### Step 2：AI 分析（選填 API Key）")
    with gr.Row():
        with gr.Column(scale=1):
            api_key_box = gr.Textbox(label="Gemini API Key", placeholder="輸入後可使用 AI 建議")
            btn_ai = gr.Button("🤖 AI 分析建議")

        with gr.Column(scale=3):
            output_ai = gr.Textbox(label="AI 決策建議", lines=8)

    hidden_ai_input = gr.Textbox(visible=False)

    gr.Markdown("### 🗺️ 餐廳地圖")
    output_map = gr.HTML()

    # ---------------- 按鈕事件 ----------------

    btn_recommend.click(
        fn=lambda p, d, a: (
            *filter_and_recommend(df_info, p, d, a),
            create_folium_map(df_map)._repr_html_()
        ),
        inputs=[price_select, distance_select, available_toggle],
        outputs=[output_df, output_list, hidden_ai_input, output_map]
    )

    btn_ai.click(
        fn=generate_ai_advice,
        inputs=[api_key_box, hidden_ai_input],
        outputs=output_ai
    )

demo.launch(share=True)

成功載入試算表！
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ea754bab1edc1e885e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [28]:
# --- 1. 驗證與載入 Google Sheet ---
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# --- 2. 套件匯入 ---
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import gradio as gr
import random

# --- 3. Google Sheet 設定 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1MKDi0Xs7pgmd-ZePl7bDVU-VIWodBp5fJ2RvJ13dfF4/edit?gid=0#gid=0"

WORKSHEET_LIST = {
    "地圖資料": "餐廳列表",
    "餐廳名稱": "工作表1"
}

# --- 4. 載入資料 ---
gsheets = gc.open_by_url(SHEET_URL)
sheet_info = gsheets.worksheet(WORKSHEET_LIST["餐廳名稱"]).get_all_values()
sheet_map = gsheets.worksheet(WORKSHEET_LIST["地圖資料"]).get_all_values()

df_info = pd.DataFrame(sheet_info[1:], columns=sheet_info[0])
df_map = pd.DataFrame(sheet_map[1:], columns=sheet_map[0])

df_map["lat"] = df_map["lat"].astype(float)
df_map["lng"] = df_map["lng"].astype(float)

print("成功載入試算表！")

# ==============================================================
#   功能函式
# ==============================================================

def filter_and_recommend(df, price_filter, distance_filter, is_available):
    """依使用者條件隨機推薦 3 間餐廳"""
    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    available_status = ['是'] if is_available else ['是', '否']

    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ]

    if filtered_df.empty:
        return pd.DataFrame(), "沒有符合條件的餐廳。", "", pd.DataFrame()

    chosen = filtered_df.sample(n=min(3, len(filtered_df)), replace=False)

    readable_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']}（{row['價位']} / {row['距離']}）- {row['特色描述']}"
        for i, row in chosen.iterrows()
    ])

    ai_text = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in chosen.iterrows()
    ])

    display_df = chosen[['餐廳名稱', '價位', '距離']].reset_index(drop=True)
    location_df = chosen[['餐廳名稱', 'lat', 'lng']]

    return display_df, readable_list, ai_text, location_df


def create_folium_map(location_df):
    """只顯示選出的那幾間餐廳"""
    if location_df.empty:
        return None

    m = folium.Map(
        location=[location_df["lat"].mean(), location_df["lng"].mean()],
        zoom_start=15
    )

    cluster = MarkerCluster().add_to(m)

    for _, row in location_df.iterrows():
        folium.Marker(
            location=[row["lat"], row["lng"]],
            tooltip=row["餐廳名稱"],
            popup=row["餐廳名稱"]
        ).add_to(cluster)

    return m


def process_recommend(price, distance, available):
    """整合推薦 + 地圖，只回傳本次選出的三間店家"""
    df, list_text, ai_input, loc_df = filter_and_recommend(df_info, price, distance, available)

    if loc_df.empty:
        return df, list_text, ai_input, "<p>沒有地圖資料</p>"

    map_html = create_folium_map(loc_df)._repr_html_()

    return df, list_text, ai_input, map_html


def generate_ai_advice(api_key, ai_input):
    """不使用 Gemini，避免錯誤"""
    if not ai_input:
        return "⚠️ 請先點開始推薦"

    suggestions = [
        "這些餐廳類型相當多元，可以根據你想吃的風味做選擇。",
        "建議選擇距離近的店家，節省中午時間！",
        "每一間都有自己的特色，看你今天的心情是哪種風格囉～"
    ]

    return "\n".join(random.sample(suggestions, 2))


# ==============================================================
#   Gradio UI
# ==============================================================

with gr.Blocks(title="午餐決策地圖推薦器") as demo:

    gr.Markdown("# 🍽️ 午餐決策地圖推薦器 2.0")

    with gr.Row():
        with gr.Column(scale=1):
            price_select = gr.CheckboxGroup(['$', '$$', '$$$'], label="價位")
            distance_select = gr.CheckboxGroup(['短', '中', '長'], label="距離")
            available_toggle = gr.Checkbox(label="只看今日營業", value=True)
            btn_recommend = gr.Button("🚀 產生推薦", variant="primary")
        with gr.Column(scale=2):
            output_df = gr.DataFrame(headers=["餐廳名稱", "價位", "距離"])
            output_list = gr.Textbox(lines=6, label="推薦清單")

    hidden_ai_input = gr.Textbox(visible=False)

    gr.Markdown("### 🗺️ 餐廳地圖")
    output_map = gr.HTML()

    btn_ai = gr.Button("🤖 AI 建議（無 Gemini）")
    output_ai = gr.Textbox(lines=5)

    # ---- 事件 ----

    btn_recommend.click(
        fn=process_recommend,
        inputs=[price_select, distance_select, available_toggle],
        outputs=[output_df, output_list, hidden_ai_input, output_map]
    )

    btn_ai.click(
        fn=lambda ai_input: generate_ai_advice("", ai_input),
        inputs=[hidden_ai_input],
        outputs=output_ai
    )

demo.launch(share=True)


成功載入試算表！
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9cb721bc7f04c26a42.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


debug版

In [20]:
# --- 1. Google 認證與 gspread ---
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# --- 2. 套件匯入 ---
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import gradio as gr
import random
import asyncio
import google.generativeai as genai  # Gemini

# --- 3. Google Sheet 設定 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1MKDi0Xs7pgmd-ZePl7bDVU-VIWodBp5fJ2RvJ13dfF4/edit#gid=0"
WORKSHEET_LIST = {
    "地圖資料": "餐廳列表",   # 有 lat/lng, 類別/特色
    "餐廳名稱": "工作表1"    # 有 餐廳名稱, 價位, 距離, 是否營業, 特色描述
}

# --- 4. 初始化資料 ---
def init_data():
    log = []

    log.append("📌 Step 1: Google 認證中...")
    # 認證已在上面完成
    log.append("✅ Google 認證 OK")

    log.append("📌 Step 2: 授權 gspread...")
    log.append("✅ gspread 授權成功")

    log.append("📌 Step 3: 連線試算表...")
    try:
        gsheets = gc.open_by_url(SHEET_URL)
        log.append("✅ 試算表連線成功")
    except Exception as e:
        log.append(f"❌ 連線失敗: {e}")
        return None, None, log

    log.append("📌 Step 4: 讀取兩個分頁...")
    sheet_info = gsheets.worksheet(WORKSHEET_LIST["餐廳名稱"]).get_all_values()
    sheet_map = gsheets.worksheet(WORKSHEET_LIST["地圖資料"]).get_all_values()
    log.append("➡ 已讀取資料 (info, map)")

    log.append("📌 Step 5: 建立 DataFrame...")
    df_info = pd.DataFrame(sheet_info[1:], columns=sheet_info[0])
    df_map = pd.DataFrame(sheet_map[1:], columns=sheet_map[0])
    log.append(f"➡ df_info 欄位：{list(df_info.columns)}")
    log.append(f"➡ df_map 欄位：{list(df_map.columns)}")

    log.append("📌 Step 6: 檢查經緯度格式...")
    try:
        df_map["lat"] = df_map["lat"].astype(float)
        df_map["lng"] = df_map["lng"].astype(float)
        log.append("➡ df_map: lat/lng 轉型成功")
    except Exception as e:
        log.append(f"❌ df_map 經緯度轉型失敗: {e}")
        return None, None, log

    log.append("📌 Step 7: 初始化完成 ✅")
    log.append(f"➡ df_info 筆數：{len(df_info)}, df_map 筆數：{len(df_map)}")
    return df_info, df_map, log

df_info, df_map, init_log = init_data()

# --- 5. 隨機推薦函數 ---
def filter_and_recommend(df, price_filter, distance_filter, is_available):
    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    available_status = ['是'] if is_available else ['是','否']

    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ].copy()

    if filtered_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", ""

    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.sample(n=n_recommend, replace=False)

    recommend_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']} ({row['價位']} / {row['距離']}) - {row['特色描述']}"
        for i, row in recommended_df.iterrows()
    ])

    ai_input = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in recommended_df.iterrows()
    ])

    return recommended_df, recommend_list, ai_input

# --- 6. Folium 地圖函數 ---
def create_folium_map(df_map, recommended_df=None):
    if recommended_df is not None and not recommended_df.empty:
        # 只顯示推薦店家
        df = pd.merge(recommended_df, df_map.rename(columns={'店家名稱':'餐廳名稱'}), on='餐廳名稱', how='left')
    else:
        df = df_map.copy()
        df.rename(columns={'店家名稱':'餐廳名稱'}, inplace=True)

    m = folium.Map(location=[df_map["lat"].mean(), df_map["lng"].mean()], zoom_start=14)
    marker_cluster = MarkerCluster().add_to(m)

    for _, row in df.iterrows():
        folium.Marker(
            location=[float(row["lat"]), float(row["lng"])],
            tooltip=row["餐廳名稱"],
            popup=row.get("類別/招牌品項", ""),
            icon=folium.Icon(icon="cloud")
        ).add_to(marker_cluster)

    return m._repr_html_()

# --- 7. AI 分析函數 ---
def call_gemini(api_key, ai_input):
    if not ai_input:
        return "⚠️ 尚未產生推薦結果。"
    if not api_key or api_key.strip() == "":
        return "⚠️ 尚未輸入 Gemini API Key。"
    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel("gemini-2.5-flash")
        prompt = f"你是一位午餐餐廳顧問，請根據以下推薦餐廳給我三句建議：\n{ai_input}"
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"❌ Gemini 發生錯誤: {e}"

# --- 8. Gradio wrapper ---
async def wrapper_recommend(price_filter, distance_filter, is_available):
    recommended_df, recommend_list, ai_input = filter_and_recommend(df_info, price_filter, distance_filter, is_available)
    display_df = recommended_df[['餐廳名稱', '價位', '距離']].reset_index(drop=True)
    map_html = create_folium_map(df_map, recommended_df)
    return display_df, recommend_list, ai_input, map_html

def wrapper_ai(api_key, ai_input):
    return call_gemini(api_key, ai_input)

# --- 9. Gradio 介面 ---
with gr.Blocks(title="午餐決策地圖推薦器") as demo:
    gr.Markdown("# 🍽️ 午餐決策地圖推薦器")

    with gr.Row():
        with gr.Column(scale=1):
            price_select = gr.CheckboxGroup(['$', '$$', '$$$'], label="價位過濾", value=['$', '$$'])
            distance_select = gr.CheckboxGroup(['短','中','長'], label="距離過濾", value=['短','中'])
            available_toggle = gr.Checkbox(label="只看今日營業", value=True)
            api_key_input = gr.Textbox(label="Gemini API Key", placeholder="輸入後才可使用 AI 分析建議")
            recommend_btn = gr.Button("🚀 開始推薦")

        with gr.Column(scale=3):
            gr.Markdown("### 📋 推薦結果")
            output_dataframe = gr.DataFrame(headers=["餐廳名稱","價位","距離"], label="推薦結果表")
            gr.Markdown("### 🧾 推薦清單細節")
            output_list = gr.Textbox(label="推薦清單細節", lines=10, interactive=False)

    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("### 🤖 AI 分析建議")
            output_summary = gr.Textbox(label="AI 決策建議", lines=8, interactive=False)
        with gr.Column(scale=3):
            gr.Markdown("### 🗺️ 餐廳地圖")
            output_map = gr.HTML(label="地圖顯示", elem_classes="border rounded-lg shadow-md")

    hidden_ai_input = gr.Textbox(visible=False)

    recommend_btn.click(
        fn=wrapper_recommend,
        inputs=[price_select, distance_select, available_toggle],
        outputs=[output_dataframe, output_list, hidden_ai_input, output_map]
    )

    ai_button = gr.Button("🤖 取得 AI 建議")
    ai_button.click(
        fn=wrapper_ai,
        inputs=[api_key_input, hidden_ai_input],
        outputs=output_summary
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://604e63464898978eea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [24]:
# ==========================================================
#   午餐決策地圖推薦器（DEBUG 版）
#   ✔ 加入進度狀態顯示
#   ✔ 避免 KeyError
# ==========================================================

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import gradio as gr
import google.generativeai as genai
import traceback

SHEET_URL = "https://docs.google.com/spreadsheets/d/1MKDi0Xs7pgmd-ZePl7bDVU-VIWodBp5fJ2RvJ13dfF4/edit"
WORKSHEET_LIST = {
    "地圖資料": "餐廳列表",
    "餐廳名稱": "工作表1"
}

# --- debug log ---
def log(msg, history):
    history.append(msg)
    return "\n".join(history)

# --- 初始化讀取 Google Sheet ---
def load_sheets(debug_history):
    try:
        debug_history = log("📌 Step 1: 驗證 Google 使用者...", debug_history)
        from google.colab import auth
        auth.authenticate_user()
        debug_history = log("✅ Google 認證成功", debug_history)

        debug_history = log("📌 Step 2: 取得憑證並授權 gspread...", debug_history)
        creds, _ = default()
        gc = gspread.authorize(creds)

        debug_history = log("📌 Step 3: 連線試算表...", debug_history)
        gsheets = gc.open_by_url(SHEET_URL)
        debug_history = log("✅ 試算表連線成功", debug_history)

        debug_history = log("📌 Step 4: 讀取兩個分頁...", debug_history)
        sheet_info = gsheets.worksheet(WORKSHEET_LIST["餐廳名稱"]).get_all_values()
        sheet_map = gsheets.worksheet(WORKSHEET_LIST["地圖資料"]).get_all_values()
        debug_history = log("✅ 成功讀取資料", debug_history)

        debug_history = log("📌 Step 5: 建立 DataFrame...", debug_history)
        df_info = pd.DataFrame(sheet_info[1:], columns=sheet_info[0])
        df_map = pd.DataFrame(sheet_map[1:], columns=sheet_map[0])
        debug_history = log(f"➡ df_info 欄位：{df_info.columns.tolist()}", debug_history)
        debug_history = log(f"➡ df_map 欄位：{df_map.columns.tolist()}", debug_history)

        if "lat" in df_info.columns:
            df_info["lat"] = df_info["lat"].astype(float)
        if "lng" in df_info.columns:
            df_info["lng"] = df_info["lng"].astype(float)

        debug_history = log("📌 Step 6: 檢查經緯度格式...", debug_history)

        df_map["lat"] = df_map["lat"].astype(float)
        df_map["lng"] = df_map["lng"].astype(float)
        debug_history = log("✅ df_map 經緯度轉型成功", debug_history)

        if 'lat' in df_info.columns and 'lng' in df_info.columns:
            df_info["lat"] = df_info["lat"].astype(float)
            df_info["lng"] = df_info["lng"].astype(float)
            debug_history = log("✅ df_info 經緯度轉型成功", debug_history)
        else:
            debug_history = log("⚠️ df_info 無 lat/lng 欄位，略過轉型", debug_history)

        debug_history = log(f"🎉 初始化完成！df_info: {len(df_info)} 筆, df_map: {len(df_map)} 筆", debug_history)

        return df_info, df_map, debug_history

    except Exception as e:
        debug_history = log(f"❌ 初始化失敗：{e}", debug_history)
        debug_history = log(traceback.format_exc(), debug_history)
        return pd.DataFrame(), pd.DataFrame(), debug_history

# --- 推薦 ---
def recommend(df_info, price_filter, distance_filter, is_available, debug_history):
    debug_history = log("📌 篩選餐廳中...", debug_history)

    price_list = price_filter if price_filter else df_info['價位'].unique()
    distance_list = distance_filter if distance_filter else df_info['距離'].unique()
    available_status = ['是'] if is_available else ['是','否']

    filtered = df_info[
        df_info['價位'].isin(price_list) &
        df_info['距離'].isin(distance_list) &
        df_info['是否營業'].isin(available_status)
    ]

    if filtered.empty:
        debug_history = log("⚠️ 無符合條件的餐廳！", debug_history)
        return pd.DataFrame(), "沒有餐廳", "", debug_history

    debug_history = log("📌 隨機挑選餐廳中...", debug_history)
    n = min(3, len(filtered))
    chosen = filtered.sample(n=n, replace=False)

    list_text = "\n".join([
        f"{i+1}. {row['餐廳名稱']}（{row['價位']} / {row['距離']}）- {row['特色描述']}"
        for i, row in chosen.iterrows()
    ])
    ai_input = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in chosen.iterrows()
    ])

    debug_history = log("✅ 推薦完成！", debug_history)
    return chosen[['餐廳名稱','價位','距離']], list_text, ai_input, debug_history

# --- 地圖 ---
def build_map(df_map, debug_history):
    debug_history = log("📌 建立地圖中...", debug_history)
    m = folium.Map(location=[df_map["lat"].mean(), df_map["lng"].mean()], zoom_start=14)
    cluster = MarkerCluster().add_to(m)
    for _, row in df_map.iterrows():
        folium.Marker(
            location=[row["lat"], row["lng"]],
            tooltip=row["店家名稱"],
            popup=row["類別/招牌品項"]
        ).add_to(cluster)
    debug_history = log("✅ 地圖建立完成！", debug_history)
    return m._repr_html_(), debug_history

# --- AI 建議 ---
def ai_advice(api_key, text, debug_history):
    debug_history = log("📌 AI 分析開始...", debug_history)
    if not api_key:
        return "⚠️ 未輸入 API Key", debug_history
    if not text:
        return "⚠️ 尚未產生推薦結果", debug_history

    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel("gemini-2.5-flash")
        prompt = f"請根據以下餐廳提供 2-3 句建議：{text}"
        res = model.generate_content(prompt)
        debug_history = log("✅ AI 回應完成！", debug_history)
        return res.text, debug_history
    except Exception as e:
        debug_history = log(f"❌ AI 錯誤：{e}", debug_history)
        return f"AI 錯誤：{e}", debug_history

# =============================================================
#   Gradio UI
# =============================================================
with gr.Blocks() as demo:

    debug_box = gr.Textbox(label="📣 系統訊息（debug）", lines=14)

    btn_load = gr.Button("🔄 初始化並載入資料", variant="primary")

    price_select = gr.CheckboxGroup(['$', '$$', '$$$'], label="價位")
    distance_select = gr.CheckboxGroup(['短','中','長'], label="距離")
    available_toggle = gr.Checkbox(label="今日營業", value=True)
    btn_recommend = gr.Button("🚀 產生推薦")

    out_df = gr.DataFrame(headers=["餐廳名稱","價位","距離"])
    out_list = gr.Textbox(label="推薦清單", lines=8)
    hidden_ai_input = gr.Textbox(visible=False)
    out_map = gr.HTML(label="地圖")

    api_key = gr.Textbox(label="Gemini API Key")
    btn_ai = gr.Button("🤖 AI 建議")
    out_ai = gr.Textbox(label="AI 建議", lines=6)

    # 狀態變數
    state_df_info = gr.State()
    state_df_map = gr.State()
    state_debug = gr.State([])

    # --- 按鈕事件 ---
    btn_load.click(
        load_sheets,
        inputs=[state_debug],
        outputs=[state_df_info, state_df_map, debug_box, state_debug]
    )

    btn_recommend.click(
        recommend,
        inputs=[state_df_info, price_select, distance_select, available_toggle, state_debug],
        outputs=[out_df, out_list, hidden_ai_input, debug_box, state_debug]
    )
    btn_recommend.click(
        build_map,
        inputs=[state_df_map, state_debug],
        outputs=[out_map, debug_box, state_debug]
    )

    btn_ai.click(
        ai_advice,
        inputs=[api_key, hidden_ai_input, state_debug],
        outputs=[out_ai, debug_box, state_debug]
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ec6465fa3d491946ef.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# --- 1. 驗證與載入 Google Sheet ---
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# --- 2. 套件匯入 ---
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import gradio as gr
import random
import asyncio

# --- 3. 你的 Google Sheet 設定 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1MKDi0Xs7pgmd-ZePl7bDVU-VIWodBp5fJ2RvJ13dfF4/edit?gid=0#gid=0"

WORKSHEET_LIST = {
    "地圖資料": "餐廳列表",   # 有 lat, lng, 類別, 特色
    "餐廳名稱": "工作表1"    # 有 餐廳名稱, 價位, 距離, 是否營業, 特色描述
}

TIMEZONE = "Asia/Taipei"

# --- 4. 從試算表載入資料 ---
gsheets = gc.open_by_url(SHEET_URL)

# 載入兩個分頁
sheet_info = gsheets.worksheet(WORKSHEET_LIST["餐廳名稱"]).get_all_values()
sheet_map = gsheets.worksheet(WORKSHEET_LIST["地圖資料"]).get_all_values()

# 轉為 DataFrame
df_info = pd.DataFrame(sheet_info[1:], columns=sheet_info[0])
df_map = pd.DataFrame(sheet_map[1:], columns=sheet_map[0])

# 轉型
df_map["lat"] = df_map["lat"].astype(float)
df_map["lng"] = df_map["lng"].astype(float)

print("✅ 成功載入 Google Sheet！")
print(f"餐廳資訊筆數：{len(df_info)}，地圖資料筆數：{len(df_map)}")

# --- 5. 功能函數 ---
def filter_and_recommend(df, price_filter, distance_filter, is_available):
    """根據使用者輸入篩選並隨機推薦 3 家餐廳"""
    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    available_status = ['是'] if is_available else ['是', '否']

    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ].copy()

    if filtered_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", ""

    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.sample(n=n_recommend, replace=False)

    recommend_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']} ({row['價位']} / {row['距離']}) - {row['特色描述']}"
        for i, row in recommended_df.iterrows()
    ])

    ai_input = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in recommended_df.iterrows()
    ])

    return recommended_df, recommend_list, ai_input


async def generate_summary(ai_input):
    """模擬 AI 決策摘要"""
    if not ai_input:
        return "請先點擊「開始推薦」以產生餐廳清單。"

    await asyncio.sleep(1)
    mock_summary = f"""
    **AI 助理 (模擬):** 今天的選擇是：{'、'.join([r.split('，')[0] for r in ai_input.split('、')])}。
    若想快速又省錢，建議選 {ai_input.split('、')[0].split('，')[0]}；
    若想吃得精緻一點，其他選項會提供更好的體驗。
    """
    return mock_summary.strip()


def create_folium_map(df_map, recommended_df):
    """建立 Folium 地圖，只顯示推薦餐廳"""
    if recommended_df.empty:
        return None

    # 合併推薦餐廳和地圖資料
    df = pd.merge(recommended_df, df_map, left_on="餐廳名稱", right_on="店家名稱", how="left")

    # 初始化地圖，以推薦餐廳平均座標為中心
    m = folium.Map(location=[df["lat"].mean(), df["lng"].mean()], zoom_start=15)

    for _, row in df.iterrows():
        folium.Marker(
            location=[row["lat"], row["lng"]],
            tooltip=row["餐廳名稱"],
            popup=row["類別/招牌品項"] + " / " + row["備註/特色"],
            icon=folium.Icon(color="red", icon="cutlery")
        ).add_to(m)

    return m


async def wrapper_recommend(price_filter, distance_filter, is_available):
    # 篩選與推薦
    recommended_df, recommend_list, ai_input = filter_and_recommend(
        df_info, price_filter, distance_filter, is_available
    )

    # AI 摘要
    ai_summary = await generate_summary(ai_input)

    # 地圖：只顯示推薦餐廳
    map_html = create_folium_map(df_map, recommended_df)._repr_html_() if not recommended_df.empty else ""

    # 顯示 DataFrame
    display_df = recommended_df[['餐廳名稱', '價位', '距離']].reset_index(drop=True)
    if display_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", "（無建議）", map_html

    return display_df, recommend_list, ai_summary, map_html


# --- 6. Gradio 介面 ---
with gr.Blocks(title="午餐決策地圖推薦器") as demo:
    gr.Markdown("# 🍽️ 午餐決策地圖推薦器")

    with gr.Row():
        with gr.Column(scale=1):
            price_select = gr.CheckboxGroup(['$', '$$', '$$$'], label="價位過濾", value=['$', '$$'])
            distance_select = gr.CheckboxGroup(['短', '中', '長'], label="距離過濾", value=['短', '中'])
            available_toggle = gr.Checkbox(label="只看今日營業", value=True)
            recommend_btn = gr.Button("🚀 開始推薦", elem_classes="w-full")

        with gr.Column(scale=3):  # 右側欄加寬
            gr.Markdown("### 📋 推薦結果")
            output_dataframe = gr.DataFrame(
                headers=["餐廳名稱", "價位", "距離"],
                label="推薦結果表",
                wrap=True
            )

            gr.Markdown("### 🧾 推薦清單細節")
            output_list = gr.Textbox(
                label="推薦清單細節",
                lines=10,              # ⬆️ 增加高度
                max_lines=20,
                interactive=False
            )

    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("### 🤖 AI 決策建議")
            output_summary = gr.Textbox(
                label="AI 決策顧問報告",
                lines=8,               # ⬆️ 增加高度
                max_lines=15,
                interactive=False
            )
        with gr.Column(scale=3):
            gr.Markdown("### 🗺️ 餐廳地圖")
            output_map = gr.HTML(label="地圖顯示", elem_classes="border rounded-lg shadow-md")

    recommend_btn.click(
        fn=wrapper_recommend,
        inputs=[price_select, distance_select, available_toggle],
        outputs=[output_dataframe, output_list, output_summary, output_map]
    )

    gr.Markdown("---")
    gr.Markdown("✅ 已連接 Google Sheet，地圖與推薦功能可直接使用。")

demo.launch(share=True)

✅ 成功載入 Google Sheet！
餐廳資訊筆數：21，地圖資料筆數：12
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://49b5ec17b4381cf4ca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
# --- 1. 驗證與載入 Google Sheet ---
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# --- 2. 套件匯入 ---
import pandas as pd
import folium
import gradio as gr
import random
import asyncio
import math

# --- 3. Google Sheet 設定 ---
SHEET_URL = "https://docs.google.com/spreadsheets/d/1MKDi0Xs7pgmd-ZePl7bDVU-VIWodBp5fJ2RvJ13dfF4/edit?gid=0#gid=0"

WORKSHEET_LIST = {
    "地圖資料": "餐廳列表",   # 有 lat, lng, 類別, 特色
    "餐廳名稱": "工作表1"    # 有 餐廳名稱, 價位, 距離, 是否營業, 特色描述
}

# --- 4. 從試算表載入資料 ---
gsheets = gc.open_by_url(SHEET_URL)
sheet_info = gsheets.worksheet(WORKSHEET_LIST["餐廳名稱"]).get_all_values()
sheet_map = gsheets.worksheet(WORKSHEET_LIST["地圖資料"]).get_all_values()

df_info = pd.DataFrame(sheet_info[1:], columns=sheet_info[0])
df_map = pd.DataFrame(sheet_map[1:], columns=sheet_map[0])
df_map["lat"] = df_map["lat"].astype(float)
df_map["lng"] = df_map["lng"].astype(float)

print("✅ 成功載入 Google Sheet！")
print(f"餐廳資訊筆數：{len(df_info)}，地圖資料筆數：{len(df_map)}")

# --- 5. 距離計算與分類 ---
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

def classify_distance(lat_user, lng_user, lat_rest, lng_rest):
    d = haversine(lat_user, lng_user, lat_rest, lng_rest)
    if d <= 0.5:
        return "短"
    elif d <= 2:
        return "中"
    else:
        return "長"

def update_distance_column(df_info, df_map, user_lat, user_lng):
    # 只取 lat/lng 欄位並改名
    df_map_sel = df_map[['店家名稱','lat','lng']].copy()
    df_map_sel.rename(columns={'店家名稱':'餐廳名稱'}, inplace=True)

    df = df_info.merge(df_map_sel, on="餐廳名稱", how="left")
    df['距離'] = df.apply(lambda row: classify_distance(user_lat, user_lng, row['lat'], row['lng']), axis=1)

    # 批量更新 Google Sheet
    worksheet_info = gc.open_by_url(SHEET_URL).worksheet(WORKSHEET_LIST["餐廳名稱"])
    distance_values = [[d] for d in df['距離']]
    worksheet_info.update(f"C2:C{len(df)+1}", distance_values)

    return df

# --- 6. 篩選與推薦 ---
def filter_and_recommend(df, price_filter, distance_filter, is_available):
    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    available_status = ['是'] if is_available else ['是','否']

    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ].copy()

    if filtered_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", ""

    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.sample(n=n_recommend, replace=False)

    recommend_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']} ({row['價位']} / {row['距離']}) - {row['特色描述']}"
        for i, row in recommended_df.iterrows()
    ])

    ai_input = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in recommended_df.iterrows()
    ])

    return recommended_df, recommend_list, ai_input

# --- 7. AI 摘要模擬 ---
async def generate_summary(ai_input):
    if not ai_input:
        return "請先點擊「開始推薦」以產生餐廳清單。"
    await asyncio.sleep(0.5)
    mock_summary = f"""
    **AI 助理 (模擬):** 推薦餐廳有：{'、'.join([r.split('，')[0] for r in ai_input.split('、')])}。
    若想快速又省錢，建議選 {ai_input.split('、')[0].split('，')[0]}；
    想精緻用餐可選其他選項。
    """
    return mock_summary.strip()

# --- 8. Folium 地圖 ---
def create_folium_map(df_map, recommended_df):
    if recommended_df.empty:
        return None
    # 只取需要的欄位並改名
    df_map_sel = df_map[['店家名稱', '類別/招牌品項', '備註/特色', 'lat', 'lng']].copy()
    df_map_sel.rename(columns={'店家名稱':'餐廳名稱'}, inplace=True)

    # merge
    df = pd.merge(recommended_df, df_map_sel, on="餐廳名稱", how="left")
    df = df.dropna(subset=['lat','lng'])
    if df.empty:
        return None
    m = folium.Map(location=[df["lat"].mean(), df["lng"].mean()], zoom_start=15)
    for _, row in df.iterrows():
        folium.Marker(
            location=[row["lat"], row["lng"]],
            tooltip=row["餐廳名稱"],
            popup=row["類別/招牌品項"] + " / " + row["備註/特色"],
            icon=folium.Icon(color="red", icon="cutlery")
        ).add_to(m)
    return m

# --- 9. Gradio wrapper ---
def wrapper_recommend(price_filter, distance_filter, is_available, user_lat, user_lng):
    df_with_distance = update_distance_column(df_info, df_map, user_lat, user_lng)
    recommended_df, recommend_list, ai_input = filter_and_recommend(
        df_with_distance, price_filter, distance_filter, is_available
    )
    # 用 asyncio.run 呼叫 async 模擬 AI 摘要
    ai_summary = asyncio.run(generate_summary(ai_input))
    map_html = create_folium_map(df_map, recommended_df)._repr_html_() if not recommended_df.empty else ""
    display_df = recommended_df[['餐廳名稱','價位','距離']].reset_index(drop=True)
    if display_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", "（無建議）", map_html
    return display_df, recommend_list, ai_summary, map_html

# --- 10. Gradio 介面 ---
with gr.Blocks(title="午餐決策地圖推薦器") as demo:
    gr.Markdown("# 🍽️ 午餐決策地圖推薦器")

    with gr.Row():
        with gr.Column(scale=1):
            price_select = gr.CheckboxGroup(['$', '$$', '$$$'], label="價位過濾", value=['$', '$$'])
            distance_select = gr.CheckboxGroup(['短','中','長'], label="距離過濾", value=['短','中'])
            available_toggle = gr.Checkbox(label="只看今日營業", value=True)
            user_lat = gr.Number(label="目前緯度", value=25.0329)
            user_lng = gr.Number(label="目前經度", value=121.5303)
            recommend_btn = gr.Button("🚀 開始推薦")

        with gr.Column(scale=3):
            output_dataframe = gr.DataFrame(headers=["餐廳名稱","價位","距離"], label="推薦結果表")
            output_list = gr.Textbox(label="推薦清單細節", lines=10, interactive=False)
            output_summary = gr.Textbox(label="AI 決策顧問報告", lines=8, interactive=False)
            output_map = gr.HTML(label="地圖顯示")

    recommend_btn.click(
        fn=wrapper_recommend,
        inputs=[price_select, distance_select, available_toggle, user_lat, user_lng],
        outputs=[output_dataframe, output_list, output_summary, output_map]
    )

demo.launch(share=True)


✅ 成功載入 Google Sheet！
餐廳資訊筆數：12，地圖資料筆數：12
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4b501a7cbd07e0377a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
